In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# from geojson import GeometryCollection

In [ ]:
# Point data to geoJSON
import random
import sqlite3

df_red = pd.read_csv("redlist_assessments.csv")

# Assume: RedList scientific correspond to UBC species
red_sciNames = df_red.scientificName

def some(x, n):
    return x.loc[random.sample(x.index, n)]

def addCommonNames(geo_df):
    
    # df_commons['common'] = df_commons[['language', 'vernacular_name']].apply(lambda x: ':'.join(x), axis=1)
    # df_commons = df_commons.groupby(['tsn'])['common'].apply(', '.join).reset_index()
    db = sqlite3.connect('ITIS.sqlite')
    df_commons = pd.read_sql_query("SELECT * from vernaculars", db)
    df_species = pd.read_sql_query("SELECT * from longnames", db)

    df_commons = df_commons[df_commons['language'] == "English"]
    df_commons = df_commons.rename({'vernacular_name': 'common'}, axis=1)
    df_commons = df_commons[['tsn', 'common']]
    
    df_species = df_species[df_species['completename'].isin(geo_df.species)]
    df_commons = df_commons[df_commons['tsn'].isin(df_species.tsn)]
    
    df_commons = df_commons.groupby(['tsn'])['common'].apply(', '.join).reset_index()
    df_species = df_species[df_species['tsn'].isin(df_commons.tsn)]

    df_common_species = df_species.merge(df_commons, on="tsn")
    df_common_species = df_common_species.rename({'completename': 'species'}, axis=1) 
    df_common_species = df_common_species.drop(['tsn'], 1)

    species = set(geo_df.species.unique())
    species_in_itis = set(df_common_species.species.unique())
    diff = list(species - species_in_itis)
    diff = {df_common_species.columns[0]: diff, df_common_species.columns[1]: 'unkown'}
    df = pd.DataFrame.from_dict(diff)
    df_species = pd.concat([df_common_species, df], sort = False)

    geo_df = geo_df.merge(df_species, on="species")
    return geo_df

def redList(x, red_sciNames):
    if (x['species'] in red_sciNames.values):
        return df_red.loc[red_sciNames[red_sciNames == x['species']].index[0]].redlistCategory
    else:
        return float('nan')

def pointCSVtoJSON(filename, num_rows=0):
    df = pd.read_csv(filename, delimiter="\t", low_memory=False)
    if (num_rows):
        df = some(df, num_rows)
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    
    # gbif_geo_df = gbif_geo_df[gbif_geo_df['species'].isin(red_sciNames)]
#     geo_df['redList'] = geo_df.apply(lambda x: int(x['species']in red_sciNames.values), 1).values
    geo_df['redList'] = geo_df.apply(lambda x: redList(x, red_sciNames), 1).values
    
    
    # Common Names ITIS
    # # Get names of all tables
    # c = conn.cursor()
    # c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # print(c.fetchall())
    
    geo_df = addCommonNames(geo_df)
    geo_df.to_file("leaflet/" + filename.split('.')[0] + '.geojson', driver="GeoJSON")
    
    return geo_df

In [ ]:
geo_df = pointCSVtoJSON('gbif_tot.csv', 2000)

In [ ]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
# str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = gpd.read_file("MVSEI2014/MVSEI2014.shp")

str_map = fix_crs(str_map)

In [ ]:
# # GeoJSON does not support multipolygon. Doesn't work
# str_map.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")

# # fiona doesn't work
# import fiona
# import json

# with fiona.open('ecological_reserves/BC_Eco_Reserves.shp') as source:
#     records = list(source)
# geo_json = {"type": "FeatureCollection","features": records}
# with open('leaflet/UBC_poly.geojson', 'w') as fp:
#     json.dump(geo_json, fp)

In [ ]:
# Convert multipolygon to single polygons

In [ ]:
geom_series = str_map.geometry

In [ ]:
def geom_apply(x):
    try:
        return list(x)
    except TypeError:
        return [x]

In [ ]:
geom_series = geom_series.apply(geom_apply)

In [ ]:
geom_series = geom_series.apply(pd.Series).stack()

In [ ]:
# # Quick but loses properties
# list_poly = list(geom_series)
# geoms = GeometryCollection(list_poly)
# geo_file = geojson.dumps(geoms)
# with open("leaflet/UBC_poly.geojson", "w") as text_file:
#     text_file.write(geo_file)

In [ ]:
# Careful! Deep copy required here to avoid chaining

df = pd.DataFrame(columns=str_map.columns)
for ind, poly in geom_series.iteritems():
    curr_row = str_map.loc[ind[0]].copy(deep=True)
    curr_row['geometry'] = poly
    df = df.append(curr_row)

df_gpd = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'})
df_gpd.to_file("leaflet/SEI.geojson", driver="GeoJSON")

In [ ]:
df_gpd.columns

In [18]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
# str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = gpd.read_file("MVSEI2014/MVSEI2014.shp")

str_map = fix_crs(str_map)

In [29]:
str_map.columns

Index([u'SourceName', u'SourceDate', u'Jurisdicti',   u'Location',
       u'Classifica', u'TEM_PolyNb', u'SEI_PolyNb',   u'SmplType',
           u'PlotNo',   u'ProjType',    u'Proj_ID',     u'EcoMap',
           u'EcoSec',   u'BGC_Unit',    u'SEDec_1',     u'SECl_1',
        u'SEsubcl_1',   u'Strct_S1', u'StrctMod_1',   u'Stand_A1',
          u'SEDec_2',     u'SECl_2',  u'SEsubcl_2',   u'Strct_S2',
       u'StrctMod_2',   u'Stand_A2',    u'SEDec_3',     u'SECl_3',
        u'SEsubcl_3',   u'Strct_S3', u'StrctMod_3',   u'Stand_A3',
        u'Microsite', u'Condition_', u'ConditionN', u'Condition1',
       u'Conditio_1', u'Conditio_2', u'Conditio_3',  u'Disturb_1',
        u'Disturb_2',  u'Disturb_3',  u'Disturb_4',    u'Context',
        u'ContextNo',  u'WSize_SE1',   u'Size_SE1', u'SizeNo_SE1',
        u'WSize_SE2',   u'Size_SE2', u'SizeNo_SE2',  u'WSize_SE3',
         u'Size_SE3', u'SizeNo_SE3', u'QualityNo_', u'WQuality_S',
       u'QualityNo1', u'WQuality_1', u'QualityN_1', u'WQuality

In [34]:
str_map.Location

0           Campbell Valley
1           Campbell Valley
2           Campbell Valley
3           Campbell Valley
4           Campbell Valley
5           Campbell Valley
6           Campbell Valley
7           Campbell Valley
8           Campbell Valley
9           Campbell Valley
10          Campbell Valley
11          Campbell Valley
12          Campbell Valley
13          Campbell Valley
14          Campbell Valley
15          Campbell Valley
16          Campbell Valley
17          Campbell Valley
18          Campbell Valley
19          Campbell Valley
20          Campbell Valley
21          Campbell Valley
22          Campbell Valley
23          Campbell Valley
24          Campbell Valley
25                     None
26                     None
27                     None
28                     None
29                     None
                ...        
24729    Indian Arm/Seymour
24730    Indian Arm/Seymour
24731    Indian Arm/Seymour
24732    Indian Arm/Seymour
24733    Indian Arm/

In [9]:
str_map = str_map[['SEI_PolyNb', 'Comp1Lgnd', 'geometry']]

In [10]:
str_map

,SEI_PolyNb,Comp1Lgnd,geometry
0,100144,Non SE/ME,POLYGON ((-122.6318513616789 49.01550396419621...
1,100014,Mature Forest (ME),"POLYGON ((-122.6470110377009 49.0149060233681,..."
2,100016,Non SE/ME,"POLYGON ((-122.632452575422 49.01479915890732,..."
3,100017,Non SE/ME,POLYGON ((-122.6276900531602 49.01379691195537...
4,100018,Non SE/ME,POLYGON ((-122.6252338228606 49.01378475880124...
5,100019,Non SE/ME,POLYGON ((-122.6318731085676 49.01041282010838...
6,100065,Non SE/ME,POLYGON ((-122.6524302088969 49.01010302373133...
7,100071,Old Field,POLYGON ((-122.6272076489002 49.01208481971054...
8,100072,Young Forest,POLYGON ((-122.6274570456395 49.01117806021158...
9,100073,Non SE/ME,POLYGON ((-122.6339485300425 49.01087824194023...


In [11]:
str_map.to_file("leaflet/SEI.geojson", driver="GeoJSON")

/Users/raghav/anaconda2/envs/jupy2/lib/python2.7/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


GeometryTypeValidationError: Record's geometry type does not match collection schema's geometry type: 'MultiPolygon' != 'Polygon'

In [14]:
def flatten_gdf_geometry(gdf, geom_type):
    geometry = gdf.geometry
    flattened_geometry = []

    flattened_gdf = gpd.GeoDataFrame()

    for geom in geometry:
        if geom.type in ['GeometryCollection', 'MultiPoint', 'MultiLineString', 'MultiPolygon']:
            for subgeom in geom:
                if subgeom.type==geom_type:
                    flattened_geometry.append(subgeom)
        else:
            if geom.type==geom_type:
                flattened_geometry.append(geom)

    flattened_gdf.geometry=flattened_geometry

    return flattened_gdf

In [15]:
new_df = flatten_gdf_geometry(str_map, 'Polygon')